In [187]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

from utils import export_image
from utils import map_image

roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").geometry().dissolve()

months = ee.List.sequence(1, 12)
years = ee.List.sequence(1985, 2019)

# mature_mask = ee.Image("projects/ee-ana-zonia/assets/mature_mask").clip(roi)
ecoregions = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi)
                .map(lambda feature: feature.intersection(roi)))

### MODIS

Calculate mean AET from MODIS and calculate CWD from the precipitation values as in Celso

Since MODIS only goes back to 2000, for now we are stuck with a fixed value for ET


In [160]:
# Function to calculate mean AET and add year property
# select for mature forests since the values can be put off by deforestation (causes lower ET)

start = "2002-01-01"
end = "2019-12-31"

modis = ee.ImageCollection("MODIS/061/MOD16A2GF") \
         .filterDate(start, end) \
         .select('ET', 'ET_QC').map(lambda image : image.clip(roi)) \
              .map(lambda image: image.reduceResolution(ee.Reducer.median(), bestEffort=True, maxPixels=1024) \
                                       .reproject(crs='EPSG:4326', scale=10000))

In [117]:
# code sourced from https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/
def bitwise_extract(input, from_bit, to_bit):
   mask_size = ee.Number(1).add(to_bit).subtract(from_bit)
   mask = ee.Number(1).leftShift(mask_size).subtract(1)
   return input.rightShift(from_bit).bitwiseAnd(mask)

def apply_QA_mask (image):
    QA = image.select('ET_QC')
    ET = image.select('ET').multiply(0.0125)  # multiply by the scale 0.1, divide by 8 to get daily info
    cloud_mask = bitwise_extract(QA, 3, 4).lte(0)
    qa_mask = bitwise_extract(QA, 5, 7).lte(1)
    mask = cloud_mask.And(qa_mask)
    return ET.updateMask(mask).set('system:time_start', image.get('system:time_start'))

# mask quality of pixels
modis_masked = modis.map(apply_QA_mask).map(lambda image:image.updateMask(mature_mask))

# Loop through the months and filter images
def monthly_et(month):
    eight_day_images = modis_masked.select('ET').filter(ee.Filter.calendarRange(month, month, 'month'))
    month_et = eight_day_images.median().multiply(30).reduceRegions(ecoregions, \
                                                                      reducer = ee.Reducer.median(), scale = 10000)
    month_et = month_et.reduceToImage(['month_et'], ee.Reducer.first())
    return month_et.set('month', month)

monthly_et_imgcol = ee.ImageCollection.fromImages(months.map(monthly_et))

yearlist = range(2002, 2019) # Generate a list of years from 1985 to 2019
monthlist = range(1, 12)
new_band_names = []
for year in yearlist:
    for month in monthlist:
        new_band_names = [f'mcwd_{month}_{year}']
monthly_et_img = monthly_et_imgcol.toBands().rename(new_band_names)

### Terraclim and seasonality

Bring temperature and precipitation and calculate seasonality

In [197]:
terraclim = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
              .filterDate('1985-01-01', '2019-12-31') \
              .map(lambda image : image.clip(roi)) \
              .map(lambda image: image.reduceResolution(ee.Reducer.median(), bestEffort=True, maxPixels=1024) \
                                       .reproject(crs='EPSG:4326', scale=10000))

maxtemp = terraclim.select('tmmx').map(lambda image: image.multiply(0.1))
mintemp = terraclim.select('tmmn').map(lambda image: image.multiply(0.1))
radiation = terraclim.select('srad').map(lambda image: image.multiply(0.1))
prec = terraclim.select('pr')

In [182]:


def cwd_monthly(current_image, previous_dict):
    current_image = ee.Image(current_image)
    previous_dict = ee.Dictionary(previous_dict)
    previous_image = ee.Image(previous_dict.get('result_image'))
    result_image = previous_image.add(current_image).subtract(100)
    result_image = result_image.where(result_image.gt(0), 0) \
                                   .set('system:time_start', current_image.get('system:time_start'))
    return ee.Dictionary({
        'result_image': ee.Image(result_image).toFloat(),
        'cwd_list': ee.List(previous_dict.get('cwd_list')).add(result_image)})

# Use iterate to apply the function to each image
cwd_dict = ee.Dictionary(prec.iterate(cwd_monthly, ee.Dictionary({'result_image': ee.Image(0).toFloat(),
                                                               'cwd_list': ee.List([])})))
cwd_list = ee.List(cwd_dict.get('cwd_list'))
# cwd_list = cwd_list.map(lambda image: image.toFloat())

def mcwd_yearly(year):
    # Calculate the start and end indices for the current year
    start = ee.Number(year).subtract(1985).multiply(12)
    end = start.add(12)
    # Slice the list to get the images for the current year
    yr = ee.List(cwd_list).slice(start, end)
    cwd_a = ee.ImageCollection.fromImages(yr)
    mcwd_a = cwd_a.reduce(ee.Reducer.min())
    return mcwd_a.set('year', year)

# Map the function over the range
yearly_mcwd_img = ee.ImageCollection.fromImages(years.map(mcwd_yearly)).toBands()

yearlist = range(1985, 2020) # Generate a list of years from 1985 to 2019
new_band_names = ['mcwd_{}'.format(year) for year in yearlist] # Append 'si_' to each year
yearly_mcwd_img = yearly_mcwd_img.rename(new_band_names)


In [179]:
dataset = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').select('pr')
# .filter(ee.Filter.calendarRange(2007, 2007, 'year')
# ).reduce(ee.Reducer.mean())

vis = {
    'min': -100,
    'max': 300,
    'palette': [
        '1a3678',
        '2955bc',
        '5699ff',
        '8dbae9',
        'acd1ff',
        'caebff',
        'e5f9ff',
        'fdffb4',
        'ffe6a2',
        'ffc969',
        'ffa12d',
        'ff7c1f',
        'ca531a',
        'ff0000',
        'ab0000',
    ],
}


mean_prec = prec.reduce(ee.Reducer.mean())
sd_prec = prec.reduce(ee.Reducer.stdDev())

# def prec_anomaly(month_prec):
#     anom = month_prec.subtract(mean_prec).divide(sd_prec)
#     drought_mask = anom.lt(0)
#     return anom.updateMask(drought_mask)

# monthly_anom = prec.map(prec_anomaly)

# def anom_yearly(year):
#     # get anomaly of the driest month of the year
#     anom_year = monthly_anom.filter(ee.Filter.calendarRange(year, year, 'year')).reduce(ee.Reducer.min())
#     return anom_year.set('year', year)

# yearly_anom = ee.ImageCollection.fromImages(years.map(anom_yearly)).toBands()

# new_band_names = ['yearly_anom_{}'.format(year) for year in yearlist]
# yearly_anom = yearly_anom.rename(new_band_names)


anom = prec.first().subtract(mean_prec).divide(sd_prec)
# drought_mask = anom.lt(0)
plot = anom.updateMask(drought_mask)


m = geemap.Map()
m.center_object(roi, 4)
m.add_layer(
    anom, vis, 'Maximum Temperature'
)
m

Map(center=[-10.620466491933243, -53.18363392270694], controls=(WidgetControl(options=['position', 'transparen…

In [152]:
# https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2006GL028946

mean_prec = prec.reduce(ee.Reducer.mean())
sd_prec = prec.reduce(ee.Reducer.stdDev())

def prec_anomaly(month_prec):
    anom = month_prec.subtract(mean_prec).divide(sd_prec)
    drought_mask = anom.lt(0)
    return anom.updateMask(drought_mask)

monthly_anom = prec.map(prec_anomaly)

def anom_yearly(year):
    # get anomaly of the driest month of the year
    anom_year = monthly_anom.filter(ee.Filter.calendarRange(year, year, 'year')).reduce(ee.Reducer.min())
    return anom_year.set('year', year)

yearly_anom = ee.ImageCollection.fromImages(years.map(anom_yearly)).toBands()

# new_band_names = ['yearly_anom_{}'.format(year) for year in yearlist]
# yearly_anom = yearly_anom.rename(new_band_names)




In [195]:
def seasonality_calc(year):
    # get mean monthly precipitation for the year
    yr_prec = prec.filter(ee.Filter.calendarRange(year, year, 'year'))
    mean_prec = yr_prec.reduce(ee.Reducer.mean())
    # Calculate absolute deviations of monthly precipitation from mean
    deviations = yr_prec.map(lambda month:month.subtract(mean_prec).abs())
    # Calculate sum of absolute deviations for each month
    sum_deviations = deviations.reduce(ee.Reducer.sum())
    # Calculate total annual precipitation
    total_annual_prec = yr_prec.reduce(ee.Reducer.sum())
    # Calculate Seasonality Index (SI)
    seasonality_index = sum_deviations.divide(total_annual_prec)
    return seasonality_index

tst = years.map(seasonality_calc)
yearly_SI = ee.ImageCollection.fromImages(tst)

# Convert ImageCollection to single Image
yearly_SI_image = yearly_SI.toBands()
new_band_names = ['si_{}'.format(year) for year in yearlist] # Append 'si_' to each year
yearly_SI_image = yearly_SI_image.rename(new_band_names)

In [172]:
def meantemp_calc(year):
    # Filter both maxtemp and mintemp ImageCollections by year at once
    yearly_maxtemp = maxtemp.filter(ee.Filter.calendarRange(year, year, 'year')).reduce(ee.Reducer.mean())
    yearly_mintemp = mintemp.filter(ee.Filter.calendarRange(year, year, 'year')).reduce(ee.Reducer.mean())
    # Calculate the mean temperature directly
    mean_temp = yearly_maxtemp.add(yearly_mintemp).divide(2)
    return mean_temp

# Map the function over the years
mean_temp = ee.ImageCollection.fromImages(years.map(meantemp_calc)).toBands()

new_band_names = ['mean_temp_{}'.format(year) for year in yearlist] # Append 'si_' to each year
mean_temp = mean_temp.rename(new_band_names)

In [196]:
def mean_yearly(year, imgcol):
    mean_year = imgcol.filter(ee.Filter.calendarRange(year, year, 'year')).reduce(ee.Reducer.mean())
    return mean_year.set('year', year)

mean_prec = ee.ImageCollection.fromImages(years.map(prec_yearly)).toBands()

yearlist = range(1985, 2020) # Generate a list of years from 1985 to 2019
new_band_names = ['prec_{}'.format(year) for year in yearlist] # Append 'si_' to each year
mean_prec = mean_prec.rename(new_band_names)
